# Table of contents<a class="anchor" id="table"></a>

* [1 Working with RDD](#1)
* [1.1 Data Preparation and Loading](#1.1)
* [1.1.1 Creating SparkSession & SparkContext](#OneOneOne)
* [1.1.2 Read CSV files, Preprocessing, and final(formatted data) RDD for all files](#OneOneTwo)
* [1.1.2.1 Flights RDD](#1.1.2.1)
* [1.1.2.2 Airports RDD](#1.1.2.2)
* [1.1.3 Show RDD number of columns, and number of records](#1.1.3)
* [1.2 Dataset flights partitioning](#1.2)
* [1.2.1 Obtain the maximum arrival time ](#1.2.1)
* [1.2.2 Obtain the maximum minimum time ](#1.2.2)
* [1.2.3 Perform range partitioning, and display the number of records in each partition](#1.2.3)
* [1.3 Query RDD](#1.3)
* [1.3.1 Collect a total number of flights for each month for all flights](#1.3.1)
* [1.3.2 Collect the average delay for each month for all flights](#1.3.2)
* [2 Working with DataFrames](#2)
* [2.1 Data Preparation and Loading](#2.1)
* [2.1.1 Define DataFrames](#2.1.1)
* [2.1.2 Display the Scheme of DataFrames](#2.1.2)
* [2.2.1 January Flights Events with ANC airport](#2.2.1)
* [2.2.2 Average Arrival Delay From Origin to Destination](#2.2.2)
* [2.2.3 Join Query with Airports DataFrame](#2.2.3)
* [2.3 Analysis](#2.3.1)
* [2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights](#2.3.1)
* [2.3.2 Display mean arrival delay each month](#2.3.2)
* [2.3.3 Relationship between mean departure delay and mean arrival delay](#2.3.3)
* [3 RDDs vs DataFrame vs Spark SQL](#3)
* [3.1 RDD Operation](#3.1)
* [3.2 DataFrame Operation](#3.1)
* [3.3 Spark SQL Operation](#3.1)



# 1 Working with RDD<a class="anchor" id="1"></a>
## 1.1 Data Preparation and Loading<a class="anchor" id="1.1"></a>
### 1.1.1 Create SparkSession and SparkContext<a class="anchor" id="OneOneOne"></a>
[Back to top](#table)

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import os
from pyspark.sql import Window
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql import Row
import time
import numpy as np

master = "local[*]"
app_name = "Assignment 1 - Morad Abou Shadi (29799260)"
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

from pyspark import SparkContext # Spark
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')


### 1.1.2 Import CSV files and Make an RDD for all files<a class="anchor" id="OneOneTwo"></a>
[Back to top](#table)

#### 1.1.2.1 Flights RDD <a class="anchor" id="1.1.2.1"></a>
[Back to top](#table)

In [3]:
# access flights file from flights.csv
flights_raw_rdd = sc.textFile('flight*.csv')
# https://sparkbyexamples.com/apache-spark-rdd/spark-read-multiple-text-files-into-a-single-rdd/#directory (reference - Read all text files matching a pattern to single RDD)

#take the array first index of array as a string
#split the string into the array using comma separator
#https://www.w3schools.com/python/ref_string_split.asp (reference)

colNameFlights = flights_raw_rdd.take(1)[0].split(",")
print(colNameFlights)

['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']


In [4]:

flights_int_columns = ['YEAR', 'MONTH', 'DAY','DAY_OF_WEEK','FLIGHT_NUMBER']
flights_float_columns = ['DEPARTURE_DELAY','ARRIVAL_DELAY','TAXI_OUT','ELAPSED_TIME','AIR_TIME','DISTANCE','TAXI_IN','TAXI_OUT'] 

In [5]:
# remove header RDD
flights_header = flights_raw_rdd.first()
flights_rdd = flights_raw_rdd.filter(lambda x: x != flights_header)
# parsing RDD
flights_rdd_1 = flights_rdd.map(lambda x: x.split(','))

def parse_rows_flights(line):
    #create dictionary
    temp = {}
    for i in range(len(line)):
        #convert to corresponding data types
        if line[i] == '':
            line[i] = 0
        if colNameFlights[i] in flights_int_columns:
            line[i] = int(line[i])
        elif colNameFlights[i] in flights_float_columns:
            line[i] = float(line[i])
        #append key value pair to dictionary
        temp[colNameFlights[i]] = line[i]
    #convert dictionary to row
    output = Row(**temp)
    return output

flights_rdd= flights_rdd_1.map(parse_rows_flights)

#### 1.1.2.2 Airports RDD <a class="anchor" id="1.1.2.2"></a>
[Back to top](#table)

In [5]:

# access airports file
airports_raw_rdd = sc.textFile('airports.csv')

colNameAirports = airports_raw_rdd.take(1)[0].split(",")
print(colNameAirports)

['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']


In [6]:
# remove header RDD
airports_header = airports_raw_rdd.first()
airports_rdd = airports_raw_rdd.filter(lambda x: x != airports_header)

# parsing RDD
airports_rdd = airports_rdd.map(lambda x: x.split(','))

def parse_rows_airports(line):
    #create dictionary
    temp = {}
    for i in range(len(line)):
        #append key value pair to dictionary
        temp[colNameAirports[i]] = line[i]
    #convert dictionary to row
    output = Row(**temp)
    return output
airports_rdd = airports_rdd.map(parse_rows_airports)
airports_rdd.take(2)
# Example row object
# Row(YEAR=2015, MONTH=6, DAY=26, DAY_OF_WEEK=5, AIRLINE='EV', FLIGHT_NUMBER=4951, TAIL_NUMBER='N707EV', ORIGIN_AIRPORT='BHM', DESTINATION_AIRPORT='LGA', SCHEDULED_DEPARTURE='630', DEPARTURE_TIME='629', DEPARTURE_DELAY=-1.0, TAXI_OUT=13.0, WHEELS_OFF='642', SCHEDULED_TIME='155', ELAPSED_TIME=141.0, AIR_TIME=113.0, DISTANCE=866.0, WHEELS_ON='935', TAXI_IN=15.0, SCHEDULED_ARRIVAL='1005', ARRIVAL_TIME='950', ARRIVAL_DELAY=-15.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='', AIR_SYSTEM_DELAY='', SECURITY_DELAY='', AIRLINE_DELAY='', LATE_AIRCRAFT_DELAY='', WEATHER_DELAY='')
# https://sparkbyexamples.com/pyspark/pyspark-row-using-rdd-dataframe/ (reference - some information about row object)

[Row(IATA_CODE='ABE', AIRPORT='Lehigh Valley International Airport', CITY='Allentown', STATE='PA', COUNTRY='USA', LATITUDE='40.65236', LONGITUDE='-75.44040'),
 Row(IATA_CODE='ABI', AIRPORT='Abilene Regional Airport', CITY='Abilene', STATE='TX', COUNTRY='USA', LATITUDE='32.41132', LONGITUDE='-99.68190')]

### 1.1.3 Show RDD number of columns, number of records, and number of partitions <a class="anchor" id="1.1.3"></a>
[Back to top](#table)

In [7]:
# FLIGHTS
#Column count
print("Number of columns in flights csv file: " + str(len(flights_rdd.take(1)[0])))

#Row count
print("Number of rows in flights csv file: " + str(flights_rdd.count()))

#Partition count
print("Number of partitions in flights csv file: " + str(flights_rdd.getNumPartitions()))

print("##############################################")
# AIRPORTS
#Column count
print("Number of columns in airports csv file: " + str(len(airports_rdd.take(1)[0])))

#Row count
print("Number of rows in airports csv file: " + str(airports_rdd.count()))

#Partition count
print("Number of partitions in airports csv file: " + str(airports_rdd.getNumPartitions()))

Number of columns in flights csv file: 31
Number of rows in flights csv file: 582184
Number of partitions in flights csv file: 20
##############################################
Number of columns in airports csv file: 7
Number of rows in airports csv file: 322
Number of partitions in airports csv file: 2


## 1.2 Dataset Partitioning <a class="anchor" id="1.2"></a>
### 1.2.1 Obtain the maximum arrival time <a class="anchor" id="1.2.1"></a>
[Back to top](#table)

In [8]:
#MAXIMUM ARRIVAL DELAY (positive value)
arrival_delay_max = flights_rdd.max(key=lambda x: (x["ARRIVAL_DELAY"]))["ARRIVAL_DELAY"]

### 1.2.2 Obtain the minimum arrival time <a class="anchor" id="1.2.2"></a>
[Back to top](#table)

In [9]:
#MINIMUM ARRIVAL DELAY (could be negative value)
arrival_delay_min = flights_rdd.min(key=lambda x:  (x["ARRIVAL_DELAY"]))["ARRIVAL_DELAY"]

In [7]:
#Print 
print("maximum delay :", arrival_delay_max)
print("minimum delay :",arrival_delay_min)

NameError: name 'arrival_delay_max' is not defined

### 1.2.3 Perform range partitioning, and display the number of records in each partition <a class="anchor" id="1.2.3"></a>
[Back to top](#table)

Highly skewed using the column of 'ARRIVAL_DELAY' for partitioning. In most flights, it is not desirable if operations are required on other columns, and it would lead to starvation in workers with less data, but prolonged processing time in workers with more data. 


In [10]:
#define a range partitioning function
no_of_partitions= 10

#Find the size of the elements in RDD
chunk_size = (arrival_delay_max - arrival_delay_min)/no_of_partitions
range_arr = []
i = arrival_delay_min
while (len(range_arr) != 10):
    range_arr.append([i, i + chunk_size])
    i += chunk_size
#Define a range of values by key to distribute across partitions

def range_function(Key):
    for index,item in enumerate(range_arr):
        if Key >=item[0] and Key <=item[1]:
            return index

#Perform range partitioning
key_value_rdd = flights_rdd.map(lambda x : (x["ARRIVAL_DELAY"], 0))
range_partitioned_rdd = key_value_rdd.partitionBy(no_of_partitions,range_function)
# print("The number of records in each partition are ")
partitions = range_partitioned_rdd.glom().collect()

for index, partition in enumerate(partitions):
    print(f"Partition {index}: {len(partition)} records")
            

Partition 0: 564038 records
Partition 1: 16568 records
Partition 2: 1237 records
Partition 3: 196 records
Partition 4: 70 records
Partition 5: 33 records
Partition 6: 21 records
Partition 7: 12 records
Partition 8: 4 records
Partition 9: 5 records


## 1.3 Query RDD  <a class="anchor" id="1.3"></a>
### 1.3.1 Collect a total number of flights for each month <a class="anchor" id="1.3.1"></a>
[Back to top](#table)

In [13]:
month_count_rdd = flights_rdd.map(lambda x: (x['MONTH'], 0)).groupByKey().mapValues(lambda vals: len(vals)).collect()
for item in month_count_rdd:
    print("Flights in Month " + str(item[0]) + " is " + str(item[1]))

Flights in Month 1 is 47136
Flights in Month 2 is 42798
Flights in Month 3 is 50816
Flights in Month 4 is 48810
Flights in Month 5 is 49691
Flights in Month 6 is 50256
Flights in Month 7 is 52065
Flights in Month 8 is 50524
Flights in Month 9 is 46733
Flights in Month 10 is 48680
Flights in Month 11 is 46809
Flights in Month 12 is 47866


### 1.3.2 Collect the average delay for each month <a class="anchor" id="1.3.2"></a>
[Back to top](#table)

In [64]:
month_avg_delay_rdd = flights_rdd.map(lambda x: (x['MONTH'], x['ARRIVAL_DELAY'])).groupByKey().mapValues(lambda vals: np.array(list(vals))).collect()
for row in month_avg_delay_rdd:
    print("Average delay of flights in month " + str(row[0]) + " is " + str(np.average((row[1]))))

Average delay of flights in month 1 is 5.652155465037339
Average delay of flights in month 2 is 7.722627225571288
Average delay of flights in month 3 is 4.889286838790932
Average delay of flights in month 4 is 3.1355050194632246
Average delay of flights in month 5 is 4.644402406874485
Average delay of flights in month 6 is 9.534662527857371
Average delay of flights in month 7 is 6.701373283395755
Average delay of flights in month 8 is 4.652501781331645
Average delay of flights in month 9 is -0.8448847709327456
Average delay of flights in month 10 is -0.5383935907970419
Average delay of flights in month 11 is 0.8206114208805999
Average delay of flights in month 12 is 6.035244223457151


# 2 Working with DataFrame <a class="anchor" id="2"></a>
## 2.1. Data Preparation and Loading <a class="anchor" id="2.1"></a>
### 2.1.1 Define dataframes and loading scheme<a class="anchor" id="2.1.1"></a>
[Back to top](#table)

In [65]:
flightsDf = spark.read.option("header",True).option("inferSchema",True).format("csv").load("flight*.csv")
airportsDf = spark.read.option("header",True).option("inferSchema",True).format("csv").load("airports.csv")

### 2.1.2 Display the schema of the final two dataframes<a class="anchor" id="2.1.2"></a>
[Back to top](#table)

In [16]:
flightsDf.printSchema()
airportsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

## 2.2. Query Analysis <a class="anchor" id="2.2"></a>
### 2.2.1 January flight events with ANC airport <a class="anchor" id="2.2.1"></a>
[Back to top](#table)

In [42]:
filtered_flights = flightsDf.filter(col("MONTH")==1).filter(col("YEAR")==2015).filter(col("ORIGIN_AIRPORT")=='ANC')
janFlightEventsAncDf = filtered_flights.select("MONTH", "ORIGIN_AIRPORT",
"DESTINATION_AIRPORT", "Distance", "ARRIVAL_DELAY")
janFlightEventsAncDf.show()

+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|Distance|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261|           -6|
|    1|   

### 2.2.2 Average Arrival Delay From Origin to Destination <a class="anchor" id="2.2.2"></a>
[Back to top](#table)

In [59]:
janFlightEventsAncAvgDf = janFlightEventsAncDf.groupBy("ORIGIN_AIRPORT","DESTINATION_AIRPORT").agg({"ARRIVAL_DELAY": "avg"}).withColumnRenamed("avg(ARRIVAL_DELAY)", "AVERAGE_DELAY").sort('AVERAGE_DELAY', ascending=True)
janFlightEventsAncAvgDf.show()

+--------------+-------------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|
+--------------+-------------------+-------------------+
|           ANC|                ADK|              -27.0|
|           ANC|                HNL|              -20.0|
|           ANC|                MSP|             -19.25|
|           ANC|                BET| -9.090909090909092|
|           ANC|                SEA| -6.490196078431373|
|           ANC|                BRW| -4.333333333333333|
|           ANC|                OME|               -3.0|
|           ANC|                ADQ|-2.6666666666666665|
|           ANC|                CDV|                1.0|
|           ANC|                OTZ|               1.25|
|           ANC|                PHX|                2.0|
|           ANC|                DEN| 3.3333333333333335|
|           ANC|                PDX|                3.5|
|           ANC|                JNU|                5.0|
|           ANC|               

### 2.2.3 Join Query with Airports DataFrame <a class="anchor" id="2.2.3"></a>
[Back to top](#table)

In [60]:
joinedSqlDf = airportsDf.join(janFlightEventsAncAvgDf, airportsDf.IATA_CODE == janFlightEventsAncAvgDf.ORIGIN_AIRPORT,"inner")
joinedSqlDf.show()

+---------+--------------------+---------+-----+-------+--------+----------+--------------+-------------------+-------------------+
|IATA_CODE|             AIRPORT|     CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|
+---------+--------------------+---------+-----+-------+--------+----------+--------------+-------------------+-------------------+
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                BRW| -4.333333333333333|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                ADK|              -27.0|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                OME|               -3.0|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                JNU|                5.0|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|

## 2.3. Analysis <a class="anchor" id="2.3"></a>
### 2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights <a class="anchor" id="2.3.1"></a>
[Back to top](#table)

In [20]:

flightsDf.createOrReplaceTempView("flightsSQL")
dailyArrivalDelay = spark.sql('''
  SELECT DAY_OF_WEEK, AVG(ARRIVAL_DELAY) AS MeanArrivalDelay, SUM(ARRIVAL_DELAY) AS TotalTimeDelay, count(*) AS NumberOfFlights
  FROM flightsSQL
  GROUP BY DAY_OF_WEEK
  ORDER BY NumberOfFlights DESC
''')
dailyArrivalDelay.show()

+-----------+------------------+--------------+---------------+
|DAY_OF_WEEK|  MeanArrivalDelay|TotalTimeDelay|NumberOfFlights|
+-----------+------------------+--------------+---------------+
|          4| 5.684831897201573|        490186|          87683|
|          1| 5.883000999381335|        494478|          86317|
|          5| 4.715112525093624|        401638|          86253|
|          3|3.9745505431431147|        335150|          85607|
|          2| 4.391518272706391|        363262|          84449|
|          7| 4.299206488272548|        343498|          81422|
|          6| 1.813841449342257|        125750|          70453|
+-----------+------------------+--------------+---------------+



### 2.3.2 Display mean arrival delay each month <a class="anchor" id="2.3.2"></a>
[Back to top](#table)

In [21]:
MonthlyArrivalDelay = spark.sql('''
  SELECT MONTH, AVG(ARRIVAL_DELAY) AS MeanArrivalDelay, SUM(ARRIVAL_DELAY) AS TotalTimeDelay, count(*) AS NumberOfFlights
  FROM flightsSQL
  GROUP BY MONTH
  ORDER BY MeanArrivalDelay
''')
MonthlyArrivalDelay.show()


+-----+-------------------+--------------+---------------+
|MONTH|   MeanArrivalDelay|TotalTimeDelay|NumberOfFlights|
+-----+-------------------+--------------+---------------+
|    9|-0.8498676252179341|        -39484|          46733|
|   10| -0.541989784312509|        -26209|          48680|
|   11| 0.8313745860658399|         38412|          46809|
|    4|  3.173803944339603|        153044|          48810|
|    5| 4.7121097658084405|        230785|          49691|
|    8|  4.713893233866763|        235063|          50524|
|    3|  5.011173860427592|        248454|          50816|
|    1|  5.804357298474946|        266420|          47136|
|   12|   6.15837046195826|        288883|          47866|
|    7|  6.786093552465234|        348907|          52065|
|    2|  8.123906203913085|        330513|          42798|
|    6|  9.747630090727856|        479174|          50256|
+-----+-------------------+--------------+---------------+



### 2.3.3 Relationship between mean departure delay and mean arrival delay <a class="anchor" id="2.3.3"></a>
[Back to top](#table)

In [22]:
MonthlyDelay = spark.sql('''
  SELECT MONTH,  AVG(DEPARTURE_DELAY) AS MeanDeptDelay, AVG(ARRIVAL_DELAY) AS MeanArrivalDelay, SUM(ARRIVAL_DELAY) AS TotalTimeDelay, count(*) AS NumberOfFlights
  FROM flightsSQL
  GROUP BY MONTH
  ORDER BY MeanDeptDelay DESC
''')
MonthlyDelay.show()

+-----+------------------+-------------------+--------------+---------------+
|MONTH|     MeanDeptDelay|   MeanArrivalDelay|TotalTimeDelay|NumberOfFlights|
+-----+------------------+-------------------+--------------+---------------+
|    6|  13.9730063585922|  9.747630090727856|        479174|          50256|
|   12|11.821651454043728|   6.15837046195826|        288883|          47866|
|    7|11.708608758020432|  6.786093552465234|        348907|          52065|
|    2|11.620796080832823|  8.123906203913085|        330513|          42798|
|    8|10.086906141367324|  4.713893233866763|        235063|          50524|
|    1|  9.75401499511029|  5.804357298474946|        266420|          47136|
|    3| 9.718308159530178|  5.011173860427592|        248454|          50816|
|    5| 9.550310180006102| 4.7121097658084405|        230785|          49691|
|    4| 7.737554783759199|  3.173803944339603|        153044|          48810|
|   11| 6.630585898709037| 0.8313745860658399|         38412|   

# 3 RDDs vs DataFrame vs Spark SQL <a class="anchor" id="3"></a>


Implement the following queries using RDDs, DataFrames and SparkSQL separately. Log the time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the computational performance difference of these 3 approaches.

<strong>Find the MONTH and DAY_OF_WEEK, number of flights, and average delay where TAIL_NUMBER = ‘N407AS’. Note number of flights and average delay should be aggregated separately. The number of flights and average delay should be grouped by both MONTH and DAYS_OF_WEEK.</strong>

## 3.1 RDD Operation<a class="anchor" id="3.1"></a>
[Back to top](#table)

In [62]:
%%time
rddOutput = flights_rdd.filter(lambda x: x["TAIL_NUMBER"] == "N407AS")\
.groupBy(lambda x: (x["MONTH"],x["DAY_OF_WEEK"]))\
.sortByKey()\
.mapValues(lambda vals: np.array(list(vals))).collect()

for block in rddOutput:
    print("MONTH: " + str(block[0][0]) + "| DAY_OF_WEEK:  " + str(block[0][1]) + "| NumberOfFlights: " + str(len(block[1]))\
         + "| MeanDeptDelay: " + str(np.average(list(map(float, block[1][:,11]))))\
         + "| MeanArrivalDelay: " + str(np.average(list(map(float, block[1][:,22])))))

MONTH: 1| DAY_OF_WEEK:  1| NumberOfFlights: 1| MeanDeptDelay: 4.0| MeanArrivalDelay: -6.0
MONTH: 1| DAY_OF_WEEK:  2| NumberOfFlights: 2| MeanDeptDelay: 12.5| MeanArrivalDelay: 17.5
MONTH: 1| DAY_OF_WEEK:  3| NumberOfFlights: 1| MeanDeptDelay: -7.0| MeanArrivalDelay: -27.0
MONTH: 1| DAY_OF_WEEK:  5| NumberOfFlights: 2| MeanDeptDelay: -6.0| MeanArrivalDelay: -21.0
MONTH: 1| DAY_OF_WEEK:  6| NumberOfFlights: 3| MeanDeptDelay: 8.666666666666666| MeanArrivalDelay: 4.333333333333333
MONTH: 2| DAY_OF_WEEK:  1| NumberOfFlights: 2| MeanDeptDelay: -4.0| MeanArrivalDelay: -2.5
MONTH: 2| DAY_OF_WEEK:  2| NumberOfFlights: 2| MeanDeptDelay: -3.5| MeanArrivalDelay: -9.5
MONTH: 2| DAY_OF_WEEK:  3| NumberOfFlights: 2| MeanDeptDelay: -12.5| MeanArrivalDelay: -11.5
MONTH: 2| DAY_OF_WEEK:  4| NumberOfFlights: 2| MeanDeptDelay: -8.5| MeanArrivalDelay: -11.0
MONTH: 2| DAY_OF_WEEK:  5| NumberOfFlights: 1| MeanDeptDelay: -11.0| MeanArrivalDelay: -31.0
MONTH: 2| DAY_OF_WEEK:  7| NumberOfFlights: 2| MeanDeptDel

## 3.2 DataFrame Operation<a class="anchor" id="3.2"></a>
[Back to top](#table)

In [63]:
%%time
filtered_df = flightsDf.filter(col("TAIL_NUMBER")=='N407AS')
filtered_df = filtered_df.groupBy("MONTH","DAY_OF_WEEK","TAIL_NUMBER")\
.agg(count("TAIL_NUMBER").alias("NumberOfFlights"),mean("DEPARTURE_DELAY").alias("MeanDeptDelay"), mean("ARRIVAL_DELAY").alias("MeanArrivalDelay"))\
.sort("MONTH","DAY_OF_WEEK")
filtered_df.show()

+-----+-----------+-----------+---------------+-----------------+-----------------+
|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumberOfFlights|    MeanDeptDelay| MeanArrivalDelay|
+-----+-----------+-----------+---------------+-----------------+-----------------+
|    1|          1|     N407AS|              1|              4.0|             -6.0|
|    1|          2|     N407AS|              2|             12.5|             17.5|
|    1|          3|     N407AS|              1|             -7.0|            -27.0|
|    1|          5|     N407AS|              2|             -6.0|            -21.0|
|    1|          6|     N407AS|              3|8.666666666666666|4.333333333333333|
|    2|          1|     N407AS|              2|             -4.0|             -2.5|
|    2|          2|     N407AS|              2|             -3.5|             -9.5|
|    2|          3|     N407AS|              2|            -12.5|            -11.5|
|    2|          4|     N407AS|              2|             -8.5|           

## 3.3 Spark SQL OPERATION<a class="anchor" id="3.3"></a>
[Back to top](#table)

In [29]:
%%time
sparkSqlOutput = spark.sql('''
  SELECT MONTH,DAY_OF_WEEK,TAIL_NUMBER, count(*) AS NumberOfFlights, AVG(DEPARTURE_DELAY) AS MeanDeptDelay, AVG(ARRIVAL_DELAY) AS MeanArrivalDelay
  FROM flightsSQL
  WHERE TAIL_NUMBER = 'N407AS'
  GROUP BY MONTH,DAY_OF_WEEK,TAIL_NUMBER
  ORDER BY MONTH,DAY_OF_WEEK
''')
sparkSqlOutput.show()

+-----+-----------+-----------+---------------+-----------------+-----------------+
|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumberOfFlights|    MeanDeptDelay| MeanArrivalDelay|
+-----+-----------+-----------+---------------+-----------------+-----------------+
|    1|          1|     N407AS|              1|              4.0|             -6.0|
|    1|          2|     N407AS|              2|             12.5|             17.5|
|    1|          3|     N407AS|              1|             -7.0|            -27.0|
|    1|          5|     N407AS|              2|             -6.0|            -21.0|
|    1|          6|     N407AS|              3|8.666666666666666|4.333333333333333|
|    2|          1|     N407AS|              2|             -4.0|             -2.5|
|    2|          2|     N407AS|              2|             -3.5|             -9.5|
|    2|          3|     N407AS|              2|            -12.5|            -11.5|
|    2|          4|     N407AS|              2|             -8.5|           